### Script to translate from English to Portugueses using HuggingFace Transformers.
- Benchmark between CPU and GPU using M2 Pro ARM processor

In [ ]:
# script de traducao com hugging face
# ! pip install transformers sentencepiece torch
# ! pip install transformers sentencepiece torch
# ! pip install huggingface-hub
# ! huggingface-cli login
# ! pip install sacremoses


In [5]:
#
# Importando os modelos
#
from transformers import MarianMTModel, MarianTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/edmundobrown/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import nltk
nltk.data.path.append('/Users/edmundobrown/nltk_data')  # Adiciona explicitamente o caminho

# Garantir que o punkt esteja instalado
nltk.download('punkt', download_dir='/Users/edmundobrown/nltk_data')



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/edmundobrown/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Running on CPU

In [1]:
from transformers import MarianMTModel, MarianTokenizer
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

# Configurar o tokenizador manualmente para dividir em sentenças
punkt_parameters = PunktParameters()
punkt_parameters.abbrev_types = set(['mr', 'dr', 'mrs', 'etc'])
sentence_tokenizer = PunktSentenceTokenizer(punkt_parameters)

# Função para carregar o modelo e tokenizer
def load_translation_model():
    model_name = "Helsinki-NLP/opus-mt-en-roa"  # Modelo atualizado para idiomas românicos
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer

# Função para traduzir uma sentença
def translate_text(text, model, tokenizer):
    text = f">>por<< {text}"
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    translated_tokens = model.generate(**inputs)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

# Função para traduzir textos grandes divididos em sentenças
def translate_large_text(text, model, tokenizer):
    sentences = sentence_tokenizer.tokenize(text)  # Usar o tokenizador manual
    translations = []
    for sentence in sentences:
        translations.append(translate_text(sentence, model, tokenizer))
    return " ".join(translations)

# Função principal para traduzir o arquivo
def translate_file(input_file_path, output_file_path):
    # Carregar o modelo e tokenizer
    model, tokenizer = load_translation_model()
    
    # Ler o conteúdo do arquivo de entrada
    with open(input_file_path, "r", encoding="utf-8") as f:
        english_text = f.read()
    
    # Traduzir o texto
    print("Traduzindo o arquivo... Isso pode levar algum tempo.")
    translated_text = translate_large_text(english_text, model, tokenizer)
    
    # Salvar a tradução no arquivo de saída
    with open(output_file_path, "w", encoding="utf-8") as f:
        f.write(translated_text)
    print(f"Tradução concluída! Arquivo salvo em: {output_file_path}")

# Caminho do arquivo de entrada (livro em inglês) e saída (tradução em português)
input_file = "/Users/edmundobrown/Documents/PosYoga/dados/Ingles/Livros(texto)/The Yoga Sutras of Patanjali- The Book of the Spiritual Man.txt"  # Substitua pelo caminho do seu arquivo em inglês
output_file = "/Users/edmundobrown/Documents/PosYoga/dados/Ingles/Livros_traduzidos/yoga_sutras_patanjali_traduzido_HF.txt"  # Arquivo traduzido


# Executar a tradução
translate_file(input_file, output_file)






/Users/edmundobrown/opt/anaconda3/envs/Yoga/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Traduzindo o arquivo... Isso pode levar algum tempo.


model.safetensors:   0%|          | 0.00/295M [00:00<?, ?B/s]

Tradução concluída! Arquivo salvo em: /Users/edmundobrown/Documents/PosYoga/dados/Ingles/Livros_traduzidos/yoga_sutras_patanjali_traduzido_HF.txt


### Running on GPU with Metal to compare the performance


In [ ]:
# ! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


In [15]:
import torch
print("Metal disponível:", torch.backends.mps.is_available())


Metal disponível: True


In [16]:
from transformers import MarianMTModel, MarianTokenizer
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import torch

# Configurar o tokenizador manualmente para dividir em sentenças
punkt_parameters = PunktParameters()
punkt_parameters.abbrev_types = set(['mr', 'dr', 'mrs', 'etc'])
sentence_tokenizer = PunktSentenceTokenizer(punkt_parameters)

# Função para carregar o modelo e tokenizer
def load_translation_model():
    model_name = "Helsinki-NLP/opus-mt-en-roa"  # Modelo atualizado para idiomas românicos
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Mover o modelo para a GPU (MPS) se disponível
    if torch.backends.mps.is_available():
        model = model.to("mps")
        print("Modelo movido para GPU com Metal.")
    else:
        print("GPU com Metal não disponível. Usando CPU.")

    return model, tokenizer

# Função para traduzir uma sentença
def translate_text(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
    translated_tokens = model.generate(**inputs)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

# Função para traduzir textos grandes divididos em sentenças
def translate_large_text(text, model, tokenizer, device):
    sentences = sentence_tokenizer.tokenize(text)  # Usar o tokenizador manual
    translations = []
    for sentence in sentences:
        translations.append(translate_text(sentence, model, tokenizer, device))
    return " ".join(translations)

# Função principal para traduzir o arquivo
def translate_file(input_file_path, output_file_path):
    # Carregar o modelo e tokenizer
    model, tokenizer = load_translation_model()
    device = "mps" if torch.backends.mps.is_available() else "cpu"

    # Ler o conteúdo do arquivo de entrada
    with open(input_file_path, "r", encoding="utf-8") as f:
        english_text = f.read()
    
    # Traduzir o texto
    print("Traduzindo o arquivo... Isso pode levar algum tempo.")
    translated_text = translate_large_text(english_text, model, tokenizer, device)
    
    # Salvar a tradução no arquivo de saída
    with open(output_file_path, "w", encoding="utf-8") as f:
        f.write(translated_text)
    print(f"Tradução concluída! Arquivo salvo em: {output_file_path}")


# Caminho do arquivo de entrada (livro em inglês) e saída (tradução em português)
input_file = "/Users/edmundobrown/Documents/PosYoga/dados/Ingles/livros_pdf/Ingles_extraido_pdf/Autism.txt"  # Substitua pelo caminho do seu arquivo em inglês
output_file = "/Users/edmundobrown/Documents/PosYoga/dados/Ingles/Livros_traduzidos/autismo_traduzido.txt"  # Arquivo traduzido
# Executar a tradução
translate_file(input_file, output_file)


Modelo movido para GPU com Metal.
Traduzindo o arquivo... Isso pode levar algum tempo.
Tradução concluída! Arquivo salvo em: /Users/edmundobrown/Documents/PosYoga/dados/Ingles/Livros_traduzidos/autismo_traduzido.txt
